# 챗봇 실시간 구현

In [ ]:
%pip install schedule

# 디스코드 챗봇 구현

In [19]:
import discord
from discord.ext import commands
from discord.ext.commands import Bot
import os
import youtube_dl
import asyncio
import nacl
import pandas as pd
import time

import datetime

In [20]:
game = discord.Game("퐝퐝이") 

intents = discord.Intents.all()

bot = commands.Bot(command_prefix='푸앙아 ', status=discord.Status.online, activity=game, intents=intents)

In [21]:
@bot.event
async def on_ready():
    print('다음으로 로그인합니다 : {0.user}'.format(bot))

In [22]:
with open("data/puang-art.txt", "r", encoding="utf-8") as f:
    data = f.read()

In [23]:
# 이스터에그
@bot.command()
async def 푸앙이(ctx):
    await ctx.send(data)

@bot.command()
async def 애옹(ctx):
    await ctx.send("https://media.discordapp.net/attachments/844584876904677440/895539776709607454/95261-20211007-140653-000.gif")

In [24]:
# 음악
@bot.command()
async def 들어와(ctx):
    if ctx.author.voice and ctx.author.voice.channel:
        channel = ctx.author.voice.channel
        await channel.connect()
        await ctx.send("자 드가자~")
    else:
        await ctx.send("음성 채널에 먼저 들어가주세요!")

@bot.command()
async def 노래틀어줘(ctx, *, search):
    channel = ctx.author.voice.channel
    if bot.voice_clients != []:
        await ctx.send("둠칫 두둠칫! " + str(bot.voice_clients[0].channel) + "에 연결되었습니다!")

        ydl_opts = {'format': 'bestaudio'}
        FFMPEG_OPTIONS = {'before_options': '-reconnect 1 -reconnect_streamed 1 -reconnect_delay_max 5', 'options': '-vn'}
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info("ytsearch:" + search, download=False)
            URL = info['entries'][0]['formats'][0]['url']
        voice = bot.voice_clients[0]
        voice.play(discord.FFmpegPCMAudio(URL, **FFMPEG_OPTIONS))
    else:
        await channel.connect()
        await ctx.send("둠칫 두둠칫! " + str(bot.voice_clients[0].channel) + "에 연결되었습니다!")

        ydl_opts = {'format': 'bestaudio'}
        FFMPEG_OPTIONS = {'before_options': '-reconnect 1 -reconnect_streamed 1 -reconnect_delay_max 5', 'options': '-vn'}
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info("ytsearch:" + search, download=False)
            URL = info['entries'][0]['formats'][0]['url']
        voice = bot.voice_clients[0]
        voice.play(discord.FFmpegPCMAudio(URL, **FFMPEG_OPTIONS))
    await ctx.send("지금 틀고 있는 노래는 " + info['entries'][0]['title'] + "입니다!")
        
@bot.command()
async def 일시정지(ctx):
    if not bot.voice_clients[0].is_paused():
        bot.voice_clients[0].pause()
        await ctx.send("일시정지!")
    else:
        await ctx.send("이미 일시정지 되어있어요!")

@bot.command()
async def 재생(ctx):
    if bot.voice_clients[0].is_paused():
        bot.voice_clients[0].resume()
        await ctx.send("재생!")
    else:
        await ctx.send("이미 재생중이에요!")

@bot.command()
async def 정지(ctx):
    if bot.voice_clients[0].is_playing():
        bot.voice_clients[0].stop()
        await ctx.send("정지!")
    else:
        await ctx.send("아무것도 재생중이지 않아요!")

@bot.command()
async def 나가(ctx):
    if bot.voice_clients != []:
        await bot.voice_clients[0].disconnect()
        await ctx.send("흐잉...")
    else:
        await ctx.send("저는 아직 아무 채널에도 들어가있지 않아요!")

In [25]:
# 챗봇
@bot.command()
async def 대화(ctx, *, message):  
    # 푸앙 메시지를 입력하면 chat(메시지) 함수를 출력합니다.
    # 이제 답장으로 푸앙이가 말할 수 있게 되었습니다.
    await ctx.reply(cb.chat(message))


In [26]:
# 명령어가 없을 때
@bot.event
async def on_command_error(ctx, error):
    if isinstance(error, commands.CommandNotFound):
        await ctx.send("그런 명령어는 존재하지 않아요!")

# 데이터 학습 및 저장 구현

In [27]:
# 기존의 ChatBotData.csv 파일을 읽어온다.
chatbot_data = pd.read_csv('data/ChatBotData.csv')

# data\ChatBotData-Old에 ChatBotData.csv의 이름을 ChatBotData(오늘의 날짜 및 시간).csv로 변경하여 저장해주는 함수
def save_old_chatbot_data():
    today = datetime.datetime.today()
    today = today.strftime("%Y-%m-%d-%H-%M-%S")
    chatbot_data.to_csv('data/ChatBotData-Old/ChatBotData({}).csv'.format(today), index=False)


# 만약 data\ChatBotData-Old에 백업 csv 파일이 10개를 초과하면 만들어진 날짜가 가장 오래된 백업 csv파일을 삭제하는 함수
def delete_old_chatbot_data():
    file_list = os.listdir('data/ChatBotData-Old')
    if len(file_list) > 10:
        # len(file_list) - 10은 백업 csv 파일이 10개를 초과하는 만큼의 숫자
        for i in range(len(file_list) - 10):
            # 백업 csv 파일이 만들어진 날짜가 가장 오래된 파일을 삭제
            os.remove('data/ChatBotData-Old/' + file_list[i])

In [28]:
# 챗봇에게 질문과 답변을 가르치는 함수
@bot.command()
async def 가르치기(ctx):
    # 먼저 디스코드 사용자에게 질문을 채팅으로 물어봅니다.
    await ctx.send("뭐라고 물어보실 건가요?")
    # 사용자가 채팅으로 답변을 입력할 때까지 기다립니다.
    def check(m):
        return m.author == ctx.author and m.channel == ctx.channel

    try:
        msg = await bot.wait_for('message', timeout=60.0, check=check)
    except asyncio.TimeoutError:
        await ctx.send('시간이 초과되었습니다.')
    else:
        # 사용자가 입력한 질문을 question에 저장합니다.
        question = msg.content
    # 받아온 question에 대한 함께 답변을 물어봅니다.
    await ctx.send(question + "에 대한 답변은 무엇인가요?")
    # 사용자가 입력한 답변을 받아옵니다.
    
    try:
        msg = await bot.wait_for('message', timeout=60.0, check=check)
    except asyncio.TimeoutError:
        await ctx.send('시간이 초과되었습니다.')
    else:
        # 사용자가 입력한 답변을 answer에 저장합니다.
        answer = msg.content
    # 받아온 질문과 답변을 quastion,answer,날짜(사용자이름) 으로ChatBotData.csv에 추가합니다.
    today = datetime.datetime.today()
    today = today.strftime("%Y-%m-%d")
    chatbot_data.loc[len(chatbot_data)] = [question, answer, today + "(" + ctx.author.name + ")"]
    # ChatBotData.csv를 저장합니다.
    chatbot_data.to_csv('data/ChatBotData.csv', index=False)
    # 질문과 답변을 모두 불러와 추가까지 하였다면 성공적으로 추가되었다는 메시지를 출력합니다.
    await ctx.send("질문과 답변이 추가되었습니다!")

# main부분

In [29]:
# Jupyter Notebook에서는 이 코드를 실행하세요
%pip install nest_asyncio 
import nest_asyncio 
nest_asyncio.apply()

Note: you may need to restart the kernel to use updated packages.


In [30]:
# 먼저 기존의 ChatBotData.csv를 data\ChatBotData-Old에 저장합니다.
save_old_chatbot_data()

# 만약 data\ChatBotData-Old에 백업 csv 파일이 10개를 초과하면 만들어진 날짜가 가장 오래된 백업 csv파일을 삭제합니다.
delete_old_chatbot_data()



In [32]:
# chat_bot.py를 20분에 한번씩 껐다키는 코드, 별개로 디스코드 봇은 계속 실행됩니다.

import schedule
import time
from IPython.display import clear_output

def job():
    import chat_bot as cb

    # jupyter notebook에서 셀 출력을 지우는 코드입니다.
    clear_output()

# 20분에 한번씩 실행합니다.
schedule.every(20).minutes.do(job)

# main을 만들어서 schedule을 실행시킵니다.

if __name__ == "__main__":

    while True:
        schedule.run_pending()
        time.sleep(1)



[2022-12-13 00:05:54] [INFO    ] discord.client: logging in using static token
[2022-12-13 00:05:54] [INFO    ] discord.client: logging in using static token
[2022-12-13 00:05:54] [INFO    ] discord.client: logging in using static token
[2022-12-13 00:05:54] [INFO    ] discord.client: logging in using static token
[2022-12-13 00:05:54] [INFO    ] discord.client: logging in using static token
[2022-12-13 00:05:54] [INFO    ] discord.client: logging in using static token
INFO:discord.client:logging in using static token


RuntimeError: Session is closed

In [ ]:
# 주의! 다음 코드는 ChatBotData.csv의 빠른 복구를 위한 코드입니다.
'''
# 기존의 data\ChatBotData-Old\ChatBotData.csv를 읽어와 data\ChatBotData.csv에 저장합니다. 데이터를 복구하시겠습니까? 라는 질문에 사용자가 y를 입력하면 복구합니다.
def restore_old_chatbot_data():
    chatbot_data = pd.read_csv('data/ChatBotData-Old/Default/ChatBotData.csv')
    chatbot_data.to_csv('data/ChatBotData.csv', index=False)
    print("기존의 ChatBotData.csv가 복구되었습니다.")

# 복구할지 물어봅니다.
print("데이터를 복구하시겠습니까?")
answer = input("y/n : ")
if answer == "y":
    # 한 번 더 물어봅니다.
    print("진짜?")
    answer = input("y/n : ")
    if answer == "y":
        print("정말로?")
        answer = input("y/n : ")
        if answer == "y":
            print("사실 복구하기 싫죠?")
            answer = input("y/n : ")
            if answer == "y":
                print("그럼 복구하지 않겠습니다.")
            else:
                restore_old_chatbot_data()
        else:
            print("데이터 복구를 취소하였습니다.")
    else:
        print("데이터 복구를 취소하였습니다.")     
else:
    print("데이터 복구를 취소하였습니다.")
'''